In [1]:
import Basic_Risk_Assessment_Tools as brat

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import numpy as np
import datetime
import matplotlib
%matplotlib inline
import copy

In [4]:
from nsetools import Nse
from nsepy import get_history

The $funding$ $ratio$ is the ratio of the current value of assets to the present value of the liabilities.

In order to compute the present value, we need to discount the amount of the liability based on the relevant interest rate derived from the yield curve.

For simplicity, we'll assume that the yield curve is flat, and so the interest rate is the same for all horizons.

The present value of a set of liabilities $L$ where each liability $L_i$ is due at time $t_i$ is given by:

$$ PV(L) = \sum_{i=1}^{k} B(t_i) L_i$$

where $B(t_i)$ is the price of a pure discount bond that pays 1 dollar at time $t_i$

If we assume the yield curve is flat and the annual rate of interest is $r$ then $B(t)$ is given by

$$B(t) = \frac{1}{(1+r)^t}$$



In [7]:
def discount(t, ir):
    """
    Compute the price of a pure discount bond that pays $1
    at time 't' (in years) and 'ir' is the annual 
    interest rate
    
    ASSUMPTION: The yield curve is flat that is the IR is the same
    for different horizons
    """
    return (1+ir)**(-t)

In [8]:
discount(10, 0.03)

0.7440939148967249

In [9]:
# CHECK CALCULATION

0.7440939148967249*(1.03**10)

1.0

In [13]:
def present_value(liabilities, ir):
    """
    Compute the present value of a list of liabilities 
    'liabilities' are indexed by time and the values are 
    amounts of each liability returns the present value of
    the sequence
    """
    dates = liabilities.index
    discounts = discount(dates, ir)
    return (discounts*liabilities).sum()

In [12]:
# Example
liabilities = pd.Series(data=[1, 1.5, 2, 2.5], index=[3, 3.5, 4, 4.5])
liabilities

3.0    1.0
3.5    1.5
4.0    2.0
4.5    2.5
dtype: float64

Here $Liabilities$ shows that we need to pay 1million in 3 years, followed by 1.5mil in 3 and a half years and so on....

In [14]:
present_value(liabilities, ir=0.03)

6.233320315080045

In [15]:
liabilities.sum()

7.0

What this shows is that I owe $7$ $million$ in the 4 and a half years but the present value of that, less the time value of money, is $6.23332$ $million$.

In other words if we invest $6.23332$ $million$ at the current Risk Free Rate of 3% then we can comfortably pay off our liabilities of $7$ $million$ in 4 and a half year.

In [18]:
def funding_ratio(assets, liabilities, ir):
    """
    Computes the funding ratio of a series of liabilities, based on 
    an interest rate and current value of assets
    """
    return assets/present_value(liabilities, ir)

In [19]:
funding_ratio(5, liabilities, 0.03)

0.8021407126958777

Therefore, at 3% $Risk$ $Free$ $Rate$ $(RFR)$ we are $currently$ $underfunded$ to meet our liabilities in 4 and a half years unless we get greater returns on our investment.

If our $RFR$ reduced to 2% then:

In [21]:
present_value(liabilities, 0.02)

6.476428599641752

In [22]:
funding_ratio(5, liabilities, 0.02)

0.7720304366941648

we're in a worse position because the liabilities have increased to $6.47$ $million$ while our asset value is the same $5$ $million$. And at a lower $RFR$ we grow our assets at a slower pace and hence can't meet the payment schedule.

In [23]:
import ipywidgets as widgets

In [25]:
def show_funding_ratio(assets, ir):
    fr = funding_ratio(assets, liabilities, ir)
    print(f'{fr*100:.2f}%')
    
controls = widgets.interactive(show_funding_ratio,
                                   assets=widgets.IntSlider(min=1, max=10, step=1, value=5),
                                   ir=(0, .20, .01)
)
display(controls)

interactive(children=(IntSlider(value=5, description='assets', max=10, min=1), FloatSlider(value=0.1, descript…

Engaging with the widget we find to meet our $Liabilities$ in 4 and a half years is either $increase$ $the$ $Asset$ $invested$ or $get$ $greater$ $returns$ $of$ $9%$ $or$ $more$.